In [1]:
"""
create universe data
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import qis as qis
import scipy.cluster.hierarchy as sch
from dataclasses import dataclass
from typing import Tuple, List, Dict
from enum import Enum
from bbg_fetch import fetch_fundamentals, fetch_field_timeseries_per_tickers

START_DATE = pd.Timestamp('31Dec2014')  # pd.Timestamp('03Aug2001')
END_DATE = pd.Timestamp.now().normalize()



In [2]:
# define local path to excel / csv data
LOCAL_PATH = "C://Users//artur//OneDrive//analytics//qdev//resources//basket_screener//"

In [3]:
def get_universe_tickers(local_path: str, is_sample: bool = False) -> List[str]:
    if is_sample:
        df = qis.load_df_from_excel(file_name='PBA EQ', local_path=local_path, sheet_name='sample')
        tickers = df['Ticker'].to_list()
        tickers_split = [x.split(' ') for x in tickers]
        tickers = [f"{x[0].upper()} {x[1].upper()} {x[-1]}" for x in tickers_split]
        tickers = list(set(tickers))
    else:
        equity_tickers = qis.load_df_from_excel(file_name='PBA EQ', local_path=local_path, sheet_name='global').index.to_list()
        etf_tickers = load_etfs_tickers(local_path=local_path)
        tickers = etf_tickers + equity_tickers
    return tickers


def create_price_data(local_path: str = None, is_sample: bool = True) -> None:
    tickers = get_universe_tickers(local_path=local_path, is_sample=is_sample)
    price_timeseries = fetch_field_timeseries_per_tickers(tickers=tickers, field='px_last',
                                                          start_date=START_DATE, end_date=END_DATE)
    if is_sample:
        file_name = 'price_timeseries_samples'
    else:
        file_name = 'price_timeseries'
    qis.save_df_to_csv(df=price_timeseries, file_name=file_name, local_path=local_path)


def create_fundamentals_data(local_path: str = None, is_sample: bool = True) -> None:
    tickers = get_universe_tickers(local_path=local_path, is_sample=is_sample)
    fundamentals = fetch_fundamentals(tickers=tickers, fields=['security_name', 'industry_sector', 'id_isin', 'crncy',
                                                               '3mo_put_imp_vol'])
    if is_sample:
        file_name = 'fundamentals_samples'
    else:
        file_name = 'fundamentals'
    qis.save_df_to_csv(df=fundamentals, file_name=file_name, local_path=local_path)

In [4]:
# create price data
create_price_data(local_path=LOCAL_PATH)
create_fundamentals_data(local_path=LOCAL_PATH)


In [5]:
# define screener

@dataclass
class UniverseScreener:
    """
    data container for bond universe
    """
    prices: pd.DataFrame
    fundamentals: pd.DataFrame
    benchmarks: pd.DataFrame

    def __post_init__(self):
        self.prices = self.prices.asfreq('B', method='ffill')
        self.benchmarks = self.benchmarks.reindex(index=self.prices.index, method='ffill')

    def get_implied_vols(self) -> pd.Series:
        return self.fundamentals['3mo_put_imp_vol'] / 100.0

    def compute_realised_volatility(self, span: int = 13, freq: str = 'W-WED', af: float = 52.0) -> pd.Series:
        # quarterly span for vol
        returns = qis.to_returns(prices=self.prices, freq=freq, drop_first=True, is_log_returns=True)
        vol = qis.compute_ewm_vol(data=returns, span=span, af=af)
        return vol.iloc[-1, :]

    def plot_implied_vs_realised(self):
        implied = self.get_implied_vols().rename('Implied Vol')
        realised = self.compute_realised_volatility().rename('Realised Vol')
        df = pd.concat([realised, implied], axis=1).dropna()
        with sns.axes_style('darkgrid'):
            fig, ax = plt.subplots(1, 1, figsize=(8, 6))
            qis.plot_scatter(df=df,
                             full_sample_order=1,
                             fit_intercept=False,
                             title='Implied volatility vs Realised volatility',
                             alpha_format='{0:+0.2%}',
                             ax=ax)

    def get_volatility(self, span: int = 52, freq: str = 'W-WED') -> pd.Series:
        implied = self.get_implied_vols()
        realised = self.compute_realised_volatility(span=span, freq=freq)
        realised = realised.reindex(index=implied.index)
        vol = pd.Series(np.where(np.isfinite(implied), implied, realised), index=implied.index)
        return vol

    def estimate_r2_and_resid_corr(self, span: int = 52, freq: str = 'W-WED',
                                   cluster_threshold: float = 5.0
                                   ) -> pd.DataFrame:
        y = qis.to_returns(prices=self.prices, freq=freq, drop_first=True, is_log_returns=True)
        x = qis.to_returns(prices=self.benchmarks, freq=freq, drop_first=True, is_log_returns=True)
        ewm_linear_model = qis.EwmLinearModel(x=x, y=y)
        ewm_linear_model.fit(span=span, is_x_correlated=True)

        loadings = {}
        for factor in x.columns:
            loadings[factor] = ewm_linear_model.loadings[factor].iloc[-1, :]
        loadings = pd.DataFrame.from_dict(loadings, orient='columns')

        # estimate R^2
        r2_t = ewm_linear_model.get_model_ewm_r2(span=span)
        # qis.plot_time_series(df=r_2)
        r2 = r2_t.iloc[-1, :]  # .sort_values()
        residual_corr_pd, residual_avg_corr = ewm_linear_model.get_model_residuals_corrs(span=span)
        residual_corr_pd.loc[np.isfinite(residual_corr_pd.to_numpy()) == False] = 0.0
        # sns.clustermap(residual_corr_pd)

        X = residual_corr_pd.to_numpy()
        Z = sch.ward(sch.distance.pdist(X))
        # sch.dendrogram(Z)
        clusters = sch.fcluster(Z, t=cluster_threshold, criterion='distance')
        clusters = pd.Series(clusters, index=self.prices.columns)
        print(f"number of clusters: {len(clusters.unique())}")

        df = pd.concat([loadings, r2.rename('r2'), residual_avg_corr.rename('resid corr'), clusters.rename('clusters')
                        ], axis=1)
        df = df.sort_values(by='r2')
        return df

    def compute_correlations(self, tickers: List[str], span: int = 52, freq: str = 'W-WED') -> pd.DataFrame:
        returns = qis.to_returns(prices=self.prices[tickers], freq=freq, drop_first=True, is_log_returns=True)
        corrs = qis.compute_ewm_covar(a=returns.to_numpy(), span=span, is_corr=True)
        corrs = pd.DataFrame(corrs, index=tickers, columns=tickers)
        return corrs

    def compute_top_stocks(self, span: int = 52, freq: str = 'W-WED',
                           vol_span: int = 13,
                           cluster_threshold: float = 5.0,
                           top_quantile: float = 0.75
                           ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        correlation_factors = self.estimate_r2_and_resid_corr(span=span, freq=freq, cluster_threshold=cluster_threshold)
        vols = self.get_volatility(span=vol_span, freq=freq).rename('vol')

        # compute scores for all universe
        r2_score = qis.df_to_cross_sectional_score(1.0 - correlation_factors['r2'], is_sorted=True).rename('r2 score') # the more the better
        resid_corr = qis.df_to_cross_sectional_score(1.0 - correlation_factors['resid corr'], is_sorted=True).rename('resid corr score')  # the more the better
        vol = qis.df_to_cross_sectional_score(vols, is_sorted=True).rename('vol score')
        scores = pd.concat([r2_score, resid_corr, vol], axis=1)
        # select top subset
        top_scores = qis.select_top_integrated_scores(scores=scores, top_quantile=top_quantile).copy()
        # merge outputs
        all_scores = pd.concat([correlation_factors, vols], axis=1).reindex(index=top_scores.index)
        top_scores = pd.concat([top_scores, all_scores], axis=1)
        # add correlations
        corrs = self.compute_correlations(tickers=top_scores.index.to_list(), span=span, freq=freq)
        return top_scores, corrs

    def compute_top_baskets(self, span: int = 52, freq: str = 'W-WED',
                            vol_span: int = 13,
                            cluster_threshold: float = 5.0,
                            top_quantile: float = 0.75,
                            basket_size: int = 3
                            ) -> Tuple[Dict[int, List[str]], pd.DataFrame, pd.DataFrame]:
        top_scores, corrs = self.compute_top_stocks(span=span, freq=freq, vol_span=vol_span,
                                                    cluster_threshold=cluster_threshold, top_quantile=top_quantile)
        assets = corrs.columns.to_list()
        n_assets = len(assets)
        available_indices = np.full(n_assets, True)
        corrs_np = corrs.to_numpy()
        selected_baskets = {}
        for idx, asset in enumerate(corrs.columns):
            selected_assets_basket = []
            if available_indices[idx]:  # select it first to the basket
                selected_assets_basket.append(asset)
                available_indices[idx] = False
            #array_rank = np.argsort(corrs_np[:, idx]).argsort()  # ranks by smalleest corr
            #array_idx_rank = {array_rank[n]: n for n in np.arange(n_assets)}  # assign rank to idx
            #array_idx_rank = dict(sorted(array_idx_rank.items()))  # sort by rank
            array_idx_rank = np.argsort(corrs_np[:, idx])  # get indices of inreasing values
            # print(array_idx_rank)
            for ranked_idx in array_idx_rank:
                if available_indices[ranked_idx]:
                    selected_assets_basket.append(assets[ranked_idx])
                    available_indices[ranked_idx] = False
                if len(selected_assets_basket) == basket_size:
                    selected_baskets[idx+1] = selected_assets_basket
                    break
            if np.all(available_indices == False):  # all assets are taken
                break
            # print(f"idx={idx}: {selected_assets_basket}")

        return selected_baskets, top_scores, corrs

    def compute_top_baskets_min_pairs(self, span: int = 52, freq: str = 'W-WED',
                                      vol_span: int = 13,
                                      cluster_threshold: float = 5.0,
                                      top_quantile: float = 0.75,
                                      basket_size: int = 3
                                      ) -> Tuple[Dict[int, List[str]], pd.DataFrame, pd.DataFrame]:
        top_scores, corrs = self.compute_top_stocks(span=span, freq=freq, vol_span=vol_span,
                                                    cluster_threshold=cluster_threshold, top_quantile=top_quantile)
        corrs_np = corrs.to_numpy()
        corr_pairs = {}
        for row, asset1 in enumerate(corrs.columns):
            for column, asset2 in enumerate(corrs.columns):
                if column > row:
                    corr_pairs[f"{asset1}-{asset2}"] = pd.Series((asset1, asset2, corrs_np[row, column]), index=['asset1', 'asset2', 'corr'])

        # sort on last value in tuple
        corr_pairs = pd.DataFrame.from_dict(corr_pairs, orient='index').sort_values(by='corr')
        print(corr_pairs)

        selected_assets = []
        n_assets = len(corrs.columns)
        n_pairs = len(corr_pairs.index)
        available_indices = np.full(n_pairs, True)
        selected_baskets = {}
        next_basket_idx = 1
        for idx, record in enumerate(corr_pairs.to_dict('records')):  #  to_dict('records') generates list of dict
            selected_assets_basket = []
            if available_indices[idx]:  # select the pair to the basket if both assets didn't enter existing baskets
                asset1, asset2 = record['asset1'], record['asset2']
                if asset1 not in selected_assets and asset2 not in selected_assets:
                    selected_assets_basket.append(asset1)
                    selected_assets.append(asset1)
                    selected_assets_basket.append(asset2)
                    selected_assets.append(asset2)
                    available_indices[idx] = False

                # look for next fill
                for idx1 in np.arange(0, n_pairs):  # selet the tird asset if it didn't entered existing assets
                    if available_indices[idx1]:
                        asset1, asset2 = corr_pairs.iloc[idx1, 0], corr_pairs.iloc[idx1, 1]
                        if asset1 not in selected_assets:
                            selected_assets_basket.append(asset1)
                            selected_assets.append(asset1)
                            available_indices[idx1] = False
                            if len(selected_assets_basket) == basket_size:
                                selected_baskets[next_basket_idx] = selected_assets_basket
                                next_basket_idx += 1
                                break
                        if asset2 not in selected_assets:
                            selected_assets_basket.append(asset2)
                            selected_assets.append(asset2)
                            available_indices[idx1] = False
                            if len(selected_assets_basket) == basket_size:
                                selected_baskets[next_basket_idx] = selected_assets_basket
                                next_basket_idx += 1
                                break
            if len(selected_assets) == n_assets or np.all(available_indices == False):  # all assets are taken
                break
        return selected_baskets, top_scores, corrs

    def create_baskets_outputs(self, selected_baskets: Dict[int, List[str]], top_scores: pd.DataFrame) -> pd.DataFrame:
        df_tickers = {}
        df_names = {}
        industry_sector = {}
        id_isin = {}
        clusters = {}
        column_names = [f"Asset {n+1}" for n in np.arange(len(selected_baskets[list(selected_baskets.keys())[0]]))]
        for idx, basket in selected_baskets.items():
            df_tickers[f"basket {idx}"] = pd.Series(basket, index=column_names)
            rename_map = dict(zip(basket, column_names))
            df_names[f"basket {idx}"] = self.fundamentals.loc[basket, 'security_name'].rename(rename_map)
            industry_sector[f"basket {idx}"] = self.fundamentals.loc[basket, 'industry_sector'].rename(rename_map)
            id_isin[f"basket {idx}"] = self.fundamentals.loc[basket, 'id_isin'].rename(rename_map)
            clusters[f"basket {idx}"] = top_scores.loc[basket, 'clusters'].rename(rename_map)

        df_tickers = pd.DataFrame.from_dict(df_tickers, orient='index')
        df_names = pd.DataFrame.from_dict(df_names, orient='index')
        industry_sector = pd.DataFrame.from_dict(industry_sector, orient='index')
        id_isin = pd.DataFrame.from_dict(id_isin, orient='index')
        clusters = pd.DataFrame.from_dict(clusters, orient='index')
        dfs = {'BBG Tickers': df_tickers, 'Names': df_names, 'Sector': industry_sector, 'ISIN': id_isin, 'Clusters': clusters}
        dfs = pd.concat(dfs, axis=1)
        return dfs


In [6]:
def load_universe_screener(local_path: str, is_sample: bool = True) -> UniverseScreener:
    if is_sample:
        prices = qis.load_df_from_csv(file_name='price_timeseries_samples', local_path=local_path)
        fundamentals = qis.load_df_from_csv(file_name='fundamentals_samples', local_path=local_path, parse_dates=False)
    else:
        prices = qis.load_df_from_csv(file_name='price_timeseries', local_path=local_path)
        fundamentals = qis.load_df_from_csv(file_name='fundamentals', local_path=local_path, parse_dates=False)
    fundamentals = fundamentals.reindex(index=prices.columns)
    benchmarks = qis.load_df_from_csv(file_name='benchmarks_timeseries', local_path=local_path)
    universe_screener = UniverseScreener(prices=prices, fundamentals=fundamentals, benchmarks=benchmarks)
    return universe_screener

In [7]:
# create screener
screener = load_universe_screener(local_path=LOCAL_PATH, is_sample=True)

In [8]:
# create baskets
selected_baskets, top_scores, corrs = screener.compute_top_baskets_min_pairs(top_quantile=0.50, cluster_threshold=7.0)


C:\python\OptimalPortfolios\lib\site-packages\qis\models\linear\ewm_factors.py:106: RuntimeWarning: Mean of empty slice
  avg_corr = pd.Series(0.5*np.nanmean(corr - np.eye(corr.shape[0]), axis=1), index=self.y.columns)


number of clusters: 1
                                      asset1          asset2      corr
UNH US Equity-META US Equity   UNH US Equity  META US Equity -0.259739
UNH US Equity-SCHW US Equity   UNH US Equity  SCHW US Equity -0.232742
BP/ LN Equity-GLW US Equity    BP/ LN Equity   GLW US Equity -0.228186
AZN LN Equity-META US Equity   AZN LN Equity  META US Equity -0.214177
6501 JT Equity-ACN US Equity  6501 JT Equity   ACN US Equity -0.210497
...                                      ...             ...       ...
FDX US Equity-AMGN US Equity   FDX US Equity  AMGN US Equity  0.499830
RIO US Equity-PRU LN Equity    RIO US Equity   PRU LN Equity  0.519317
BP/ LN Equity-FR FP Equity     BP/ LN Equity    FR FP Equity  0.535011
BHP US Equity-PRU LN Equity    BHP US Equity   PRU LN Equity  0.579162
RIO US Equity-BHP US Equity    RIO US Equity   BHP US Equity  0.899234

[276 rows x 3 columns]


In [9]:
for key, basket in selected_baskets.items():
    print(f"{key}: {basket}")

1: ['UNH US Equity', 'META US Equity', 'SCHW US Equity']
2: ['BP/ LN Equity', 'GLW US Equity', 'AZN LN Equity']
3: ['6501 JT Equity', 'ACN US Equity', 'MRK US Equity']
4: ['AIR FP Equity', 'FDX US Equity', 'AMGN US Equity']
5: ['CMG US Equity', 'ASML NA Equity', 'BHP US Equity']
6: ['PRU LN Equity', 'DGE LN Equity', 'STZ US Equity']
7: ['SMCI US Equity', '3382 JP Equity', 'RIO US Equity']
8: ['WMT US Equity', 'FR FP Equity', 'ADBE US Equity']


In [12]:
# create outputs
screener.create_baskets_outputs(selected_baskets=selected_baskets, top_scores=top_scores)

BBG Tickers                                  \
                 Asset 1         Asset 2         Asset 3   
basket 1   UNH US Equity  META US Equity  SCHW US Equity   
basket 2   BP/ LN Equity   GLW US Equity   AZN LN Equity   
basket 3  6501 JT Equity   ACN US Equity   MRK US Equity   
basket 4   AIR FP Equity   FDX US Equity  AMGN US Equity   
basket 5   CMG US Equity  ASML NA Equity   BHP US Equity   
basket 6   PRU LN Equity   DGE LN Equity   STZ US Equity   
basket 7  SMCI US Equity  3382 JP Equity   RIO US Equity   
basket 8   WMT US Equity    FR FP Equity  ADBE US Equity   

                               Names                             \
                             Asset 1                    Asset 2   
basket 1      UnitedHealth Group Inc         Meta Platforms Inc   
basket 2                      BP PLC                Corning Inc   
basket 3                 Hitachi Ltd              Accenture PLC   
basket 4                   Airbus SE                 FedEx Corp   
basket 5  Chipotle Mexican Grill Inc            ASML Holding NV   
basket 6              Prudential PLC                 Diageo PLC   
basket 7    Super Micro Computer Inc  Seven & i Holdings Co Ltd   
basket 8                 Walmart Inc                   Valeo SE   

                                                    Sector  \
                           Asset 3                 Asset 1   
basket 1   Charles Schwab Corp/The  Consumer, Non-cyclical   
basket 2           AstraZeneca PLC                  Energy   
basket 3            Merck & Co Inc              Industrial   
basket 4                 Amgen Inc              Industrial   
basket 5             BHP Group Ltd      Consumer, Cyclical   
basket 6  Constellation Brands Inc               Financial   
basket 7             Rio Tinto PLC              Technology   
basket 8                 Adobe Inc      Consumer, Cyclical   

                                                                  ISIN  \
                         Asset 2                 Asset 3       Asset 1   
basket 1          Communications               Financial  US91324P1021   
basket 2          Communications  Consumer, Non-cyclical  GB0007980591   
basket 3              Technology  Consumer, Non-cyclical  JP3788600009   
basket 4              Industrial  Consumer, Non-cyclical  NL0000235190   
basket 5              Technology         Basic Materials  US1696561059   
basket 6  Consumer, Non-cyclical  Consumer, Non-cyclical  GB0007099541   
basket 7  Consumer, Non-cyclical         Basic Materials  US86800U3023   
basket 8      Consumer, Cyclical              Technology  US9311421039   

                                     Clusters                  
               Asset 2       Asset 3  Asset 1 Asset 2 Asset 3  
basket 1  US30303M1027  US8085131055        1       1       1  
basket 2  US2193501051  GB0009895292        1       1       1  
basket 3  IE00B4BNMY34  US58933Y1055        1       1       1  
basket 4  US31428X1063  US0311621009        1       1       1  
basket 5  NL0010273215  US0886061086        1       1       1  
basket 6  GB0002374006  US21036P1084        1       1       1  
basket 7  JP3422950000  US7672041008        1       1       1  
basket 8  FR0013176526  US00724F1012        1       1       1